In [ ]:
import matplotlib

matplotlib.use('TkAgg')
import LDP.basicDP.OLHbasic as lhb
import LDP.basicFunction.basicfunc as bf
import LDP.basicDP.LDPKVbasic as lkvb
import numpy as np
import random
import time

"""数据处理"""
# 关闭科学计数法显示
np.set_printoptions(suppress=True)

#### 读取文件
#### 构建kv对元组列表，每一行代表一个用户的kv对list

In [ ]:
k_path='../Movie/data/Movie_k.txt'
v_path = '../Movie/data/Movie_v.txt'
lable_path = '../Movie/data/Movie_lable.txt'

data_k = bf.readtxt(k_path)
data_v = bf.readtxt(v_path)
data_l = bf.readtxt(lable_path)

n = len(data_k)  # 用户数量
l = len(data_l[0])  # key域的长度
epsilon = 5

# 构建kv对元组列表，每一行代表一个用户的kv对list
kv = []

for i in range(n):
    data_k[i]=list(map(int,data_k[i]))
    tmp = zip(data_k[i], data_v[i])
    kv.append(list(tmp))
  
# 构建label标签，我已经把标签规范化为[1,l]的数字了，也就是说不需要用之前的原始标签了
label=[i+1 for i in range(l)]

## 1.采样
#### 注意！！！！要转换为和samplek相同的格式，因为hash是专为str计算的，int和float转换得到的结果不同

In [ ]:

samplekv = []
for i in range(n):
    random.seed(1)
    tmp = random.sample(kv[i], 1)
    samplekv.append(tmp[0])
samplekv_unzip = list(zip(*samplekv))
samplek = samplekv_unzip[0]

samplek

#### samplek的真实计数结果

In [ ]:
count_true = []  # 真实计数结果
for i in range(l):
    count_true.append((label[i], samplek.count(label[i])))

count_true

In [ ]:
samplekpath='../Movie/result/samplekcount.txt'
bf.savetxt(count_true,samplekpath)

## 2.用OLH查找候选集

In [ ]:
starttime = time.clock()
print("开始执行OLH...")

# 调用OLH
est=lhb.OLH_1(epsilon,samplek,n,l)

# 程序运行结束时间
endtime = time.clock()
print('运行时间:', endtime - starttime)

est

In [ ]:
# 保存采样结果
bf.savetxt(est,'../Movie/result/sampleresult.txt')

#### 生成top_k的候选集(候选集长度为2k)

In [ ]:
k=20
est.sort(key=lambda x:x[1],reverse=True)
topk_candi=est[:2*k]


#在不会出现相同key的情况下，可以转为dict，提升速度
candi_dict=dict(topk_candi)
candi_k=list(candi_dict.keys())
candi_k

In [ ]:
# 保存候选集，长度为2k
bf.savetxt(candi_k,'../Movie/result/candidate.txt')


#### 与每个用户的项集kv取交集得到用户本地项集minikv

In [ ]:
minik=[]
miniv=[]
minikv=[]
# 每个用户的交集项个数
numsize=[]
for i in range(n):
    # data_k[i]是每个用户的项集的k值
    # 将data_k[i]与候选集candi_k取交集得到tmpk
    tmpk=list(set(candi_k).intersection(set(data_k[i])))
    # 根据tmpk得到对应项的v值tmpv
    di=dict(kv[i])
    tmpv=list(map(di.get,tmpk))
    minik.append(tmpk)
    miniv.append(tmpv)
    minikv.append(list(zip(tmpk,tmpv)))
    numsize.append(len(tmpk))
    
numsize

In [ ]:
# 保存用户取交集之后的本地项集minik和miniv
bf.savetxt(minik,'../Movie/result/minik.txt')
bf.savetxt(miniv,'../Movie/result/miniv.txt')

## 3.用OLH上传交集的项数

In [ ]:
starttime = time.clock()
print("开始执行OLH...")
numsize_lable=[i for i in range(2*k+1)]
# 调用OLH
numsize_est = lhb.OLH(epsilon, numsize, n, numsize_lable)

# 程序运行结束时间
endtime = time.clock()
print('运行时间:', endtime - starttime)

numsize_est

In [ ]:
#### 获得L值

In [ ]:
d=dict(numsize_est)
s=sum(list(d.values()))
target=0.9*s
target


# todo:其实这里的L值就是前面的90%的和，等下补个函数计算一下
L=50

## 4.Padding and Sampling

In [ ]:
padlabel=[i+1 for i in range(l,l+L)]
cp_label=candi_k+padlabel
upkv=[lkvb.P_S(minikv[i],l,L) for i in range(n)]

upkv

In [ ]:

bf.savetxt(upkv,'../Movie/result/upkv.txt')
bf.savetxt(cp_label,'../Movie/result/cp_label.txt')

#### UE扰动


In [ ]:

d=2*k
eps=10
eps1=np.log((np.exp(eps)+1)/2)
eps2=eps
a=0.5
p=lkvb.p_values(eps2)
b=1-lkvb.p_values(eps1)

In [ ]:
Y=lkvb.PCKV_UE(upkv,d,L,cp_label,a,p,b)

####聚合

In [ ]:
f_k,m_k=lkvb.AEC_UE(Y,d,L,n,a,p,b)
print(f_k)
print(m_k)

In [ ]:
lable_path = '../Movie/data/Movie_id_count_avg.txt'

id_count= bf.readtxt(lable_path)
id=[]
true_count=[]
true_avg=[]
for i in candi_k:
    tmp=id_count[int(i)-1]
    id.append(tmp[0])
    true_count.append(tmp[1])
    true_avg.append(tmpv[2])
true_count

In [ ]:

bf.savetxt(id,'../Movie/result/id.txt')
bf.savetxt(true_count,'../Movie/result/true_count.txt')
bf.savetxt(true_avg,'../Movie/result/true_avg.txt')

In [ ]:
est_count=[]
for i in f_k:
    est_count.append(i*n)
bf.savetxt(est_count,'../Movie/result/est_count.txt')
